In [0]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print(fn, len(uploaded[fn]))

Saving test_data.csv to test_data.csv
test_data.csv 22382642


In [2]:
from google.colab import files
uploaded_train = files.upload()

for fn in uploaded.keys():
  print(fn, len(uploaded[fn]))

Saving train_data.csv to train_data.csv


NameError: ignored

In [0]:

for fn in uploaded.keys():
  print(fn, len(uploaded[fn]))

test_data.csv 22382642


In [0]:
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

data = pd.read_csv("train_data_400.csv")
test = pd.read_csv("test_data_400.csv")

In [19]:
t_data.pop(t_data.columns[0])
t_data.sample()

,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,user_id,action_id,time_50hz
88222,-1.388325,2.328253,-0.020834,0.983398,0.180241,-0.247492,-0.867763,0.128629,0.069464,0.071274,0.029872,11,dws_2,223


feature selection: Correlation matrix

In [2]:
action_list = ['dws', 'jog', 'ups', 'wlk','sit','std']


#user_id=1
#t_data = data.loc[(data['action_id'].str.match(action_id+"$") ) & (data['user_id'] == user_id)]

total_list = []
curve_acc = 0.5

for action_id in action_list:
  print("-"*50,"\naction_id:",action_id)
  t_data = data.loc[(data['action_id'].str.match(action_id) )]
  t_data = t_data.drop(['action_id','user_id', 'time_50hz'], axis=1)
  t_data.pop(t_data.columns[0]) #remove index


  e = t_data.T
  #e = t_data
  c = np.corrcoef(e)

  e = np.abs(c)
  print(e.shape)
  #print(b)
  #print(*e, sep="\n")
  #print(e.shape)

  curve = curve_acc
  cnt_t = 0

  target = [1]*len(e)

  print(np.around(e, decimals=3))

  for i in range(len(e)):
      cnt = 0
      for j in range(i+1, len(e)):
          if e[i][j] > 1-curve and  target[j] == 1:
              cnt+=1
              target[j] = 0
              cnt_t += cnt
  #    print(cnt, end=", ")


  print("\n", cnt_t)
  print(np.sum(target))
  for i in range(len(target)):
    if target[i]:
      print(t_data.columns[i], end=', ')
  total_list.append(target)

  print("\n","-"*50)
  
print(*total_list, sep="\n")

-------------------------------------------------- 
action_id: dws
(12, 12)
[[1.    0.036 0.106 0.714 0.092 0.089 0.065 0.022 0.011 0.084 0.039 0.075]
 [0.036 1.    0.079 0.115 0.977 0.015 0.009 0.063 0.016 0.034 0.023 0.017]
 [0.106 0.079 1.    0.135 0.085 0.005 0.005 0.048 0.019 0.002 0.017 0.019]
 [0.714 0.115 0.135 1.    0.193 0.132 0.098 0.04  0.004 0.094 0.034 0.036]
 [0.092 0.977 0.085 0.193 1.    0.045 0.008 0.077 0.002 0.03  0.017 0.005]
 [0.089 0.015 0.005 0.132 0.045 1.    0.006 0.024 0.207 0.084 0.034 0.284]
 [0.065 0.009 0.005 0.098 0.008 0.006 1.    0.079 0.324 0.046 0.08  0.02 ]
 [0.022 0.063 0.048 0.04  0.077 0.024 0.079 1.    0.065 0.019 0.019 0.064]
 [0.011 0.016 0.019 0.004 0.002 0.207 0.324 0.065 1.    0.07  0.033 0.133]
 [0.084 0.034 0.002 0.094 0.03  0.084 0.046 0.019 0.07  1.    0.073 0.124]
 [0.039 0.023 0.017 0.034 0.017 0.034 0.08  0.019 0.033 0.073 1.    0.078]
 [0.075 0.017 0.019 0.036 0.005 0.284 0.02  0.064 0.133 0.124 0.078 1.   ]]

 2
10
attitude.roll, a

Feature selection: PCA

In [34]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3
from sklearn.decomposition import PCA

def classifier(data):
    if data == "wlk":
        return 1
    elif data == "jog":
        return 2
    elif data == "std":
        return 3
    elif data == "sit":
        return 4
    elif data == "dws":
        return 5
    elif data == "ups":
        return 6
    else:
        print("FUCK")
        return -1

      
comp_count = 3

X = data.drop(['user_id', 'action_id', 'time_50hz'], axis=1)
Y = list(map(lambda x:classifier(x[:3]), data['action_id']))

pca = PCA(n_components=comp_count)
principalComponents = pca.fit_transform(X)
print(pca.explained_variance_ratio_)  
print(pca.singular_values_)  



principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3'])

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(111, projection='3d') 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_zlabel('Principal Component 3', fontsize = 15)
ax.set_title('3 Component PCA', fontsize = 20)

final["action_id"] = data.action_id.apply(lambda x:x[:3])

final.head(5)



targets = [*set(final["action_id"])]
colors = ['red', 'green', 'blue', 'black', 'yellow', 'purple']
for target, color in zip(targets,colors):
    indicesToKeep = final['action_id'] == target
    ax.scatter(final.loc[indicesToKeep, 'principal component 1']
               , final.loc[indicesToKeep, 'principal component 2']
               , final.loc[indicesToKeep, 'principal component 3']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()




ImportError: ignored

In [33]:
!pip install --upgrade matplotlib

    100% |████████████████████████████████| 12.9MB 2.4MB/s 
    100% |████████████████████████████████| 952kB 15.3MB/s 
  Found existing installation: matplotlib 2.1.2
    Uninstalling matplotlib-2.1.2:
      Successfully uninstalled matplotlib-2.1.2


In [29]:
import matplotlib
matplotlib.__version__

'2.1.2'

Feature selection: Forward selection

In [53]:
import tensorflow as tf

def classifier(data):
    if data == "wlk":
        return 1
    elif data == "jog":
        return 2
    elif data == "std":
        return 3
    elif data == "sit":
        return 4
    elif data == "dws":
        return 5
    elif data == "ups":
        return 6
    else:
        print("FUCK")
        return -1

#prepare data

data.pop(data.columns[0])
train_X = data.drop(['user_id', 'action_id', 'time_50hz'], axis=1)
train_Y = list(map(lambda x:classifier(x[:3]), data['action_id']))
class_count = len(set(train_Y))

train_Y_one_hot = tf.one_hot(train_Y, class_count)

test_X = test.drop(['user_id', 'action_id', 'time_50hz'], axis=1)
test_Y = list(map(lambda x:classifier(x[:3]), test['action_id']))

test_Y_one_hot = tf.one_hot(test_Y, class_count)


for col in train_X.columns:
  print("-"*50)
  print("Column:%s"%(col))
  learning_rate = 0.01
  #column_length = len(tot2.columns)
  column_length = 1
  #class_count = len(set(Y_class))
  x = tf.placeholder(tf.float32, [None, column_length])
  y = tf.placeholder(tf.float32, [None, class_count], name="Y")
  #use tot2

  W = tf.Variable(tf.zeros([column_length,class_count], tf.float32))
  b = tf.Variable(tf.zeros(class_count))

  pred = tf.nn.softmax(tf.matmul(x,W) + b)
  cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

  init = tf.global_variables_initializer()

  with tf.Session() as sess:

      y_one_hot_out = sess.run(train_Y_one_hot)
      test_y_one_hot_out = sess.run(test_Y_one_hot)

      sess.run(init)

      total_batch = len(train_X)
      batch_size = 400
      epoch_count = 70
      for epoch in range(epoch_count):
          avg_cost=0
          for i in range(int(total_batch / batch_size)):
              batch_xs, batch_ys = train_X[col].values[i * batch_size : (i+1)*batch_size], y_one_hot_out[i * batch_size : (i+1)*batch_size]
              batch_xs = batch_xs.reshape(batch_size, 1)
              #print(type(batch_xs), batch_xs.shape)
              _,c = sess.run([optimizer, cost], feed_dict={x: batch_xs, y:batch_ys})

              avg_cost += c / total_batch 
          
          
          if epoch % 10 == 0:
            print("epoch", "%d / %d"%(epoch, epoch_count), "cost: %.9f"%(avg_cost))

      correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y,1))
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
      print("test Accuracy:", accuracy.eval({x: test_X[col].values.reshape(len(test_X[col].values),1), y:test_y_one_hot_out}))

      print("-"*50)

--------------------------------------------------
Column:userAcceleration.z
epoch 0 / 70 cost: 0.003502843
epoch 10 / 70 cost: 0.003235725
epoch 20 / 70 cost: 0.003209892
epoch 30 / 70 cost: 0.003200190
epoch 40 / 70 cost: 0.003195120
epoch 50 / 70 cost: 0.003192007
epoch 60 / 70 cost: 0.003189903
test Accuracy: 0.20008901
--------------------------------------------------


# 새 섹션

In [0]:
a = set(data['action_id'])
a

{'dws_1',
 'dws_11',
 'dws_2',
 'jog_16',
 'jog_9',
 'sit_13',
 'sit_5',
 'std_14',
 'std_6',
 'ups_12',
 'ups_3',
 'ups_4',
 'wlk_15',
 'wlk_7',
 'wlk_8'}

In [0]:
a = [3,1,5,4,2]
a[-2]

4

In [0]:
data.columns

Index(['attitude.roll', 'attitude.pitch', 'attitude.yaw', 'gravity.x',
       'gravity.y', 'gravity.z', 'rotationRate.x', 'rotationRate.y',
       'rotationRate.z', 'userAcceleration.x', 'userAcceleration.y',
       'userAcceleration.z', 'user_id', 'action_id', 'time_50hz'],
      dtype='object')

In [0]:
set(data.loc[data['action_id']=='dws_1']['user_id'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21}

In [0]:
action_id = 'dws_1'
user_id=1
t_data = data.loc[(data['action_id'].str.match(action_id+"$") ) & (data['user_id'] == user_id)]

In [0]:
from matplotlib.mlab import find
def freq_zero_crossing(sig, fs):
    """
    Frequency estimation from zero crossing method
    sig - input signal
    fs - sampling rate
    
    return: 
    dominant period
    """
    print(len(sig))
    # Find the indices where there's a crossing
    indices = find((sig[1:] >= 0) & (sig[:-1] < 0))
    print("find(sig[1:] >= 0)", find(sig[1:] >= 0), "\nfind(sig[:-1] < 0)", find((sig[:-1] < 0)), "\nindices", indices)

    # Let's calculate the real crossings by interpolate
    crossings = [i - sig[i] / (sig[i+1] - sig[i]) for i in indices]
    print("crossings", crossings)
    
    # Let's get the time between each crossing
    # the diff function will get how many samples between each crossing
    # we divide the sampling rate to get the time between them
    delta_t = np.diff(crossings) / fs
    print("np.diff(crossings)", np.diff(crossings), "\ndelta_t", delta_t)
    
    # Get the mean value for the period
    period = np.mean(delta_t)
    
    return period

arr = [0.351138,0.351219,0.351145,0.350849,0.350859,0.350996,0.351274,0.351431,0.351272,0.35134,0.351436,0.351694,0.351589,0.351387,0.351165,0.350823,0.350802,0.351018,0.351121,0.351365,0.351928,0.352344,0.352494,0.352595,0.352503,0.352326,0.352341,0.35226,0.352284,0.352454,0.352723,0.352823,0.352897,0.353135,0.353174,0.353188,0.353286,0.353429,0.353527,0.353672,0.353703,0.353801,0.353818,0.353908,0.354166,0.354303,0.354287,0.3545,0.354549,0.354703,0.354853,0.355135,0.355325,0.355404,0.355373,0.355366,0.355478,0.355615,0.355783,0.356111,0.356491,0.356853,0.357006,0.357066,0.357087,0.357074,0.357177,0.357523,0.357816,0.357924,0.35795,0.357862,0.357735,0.357719,0.357646,0.357632,0.357714,0.35787,0.358048,0.358096,0.35799,0.358028,0.358166,0.35825,0.358422,0.358476,0.358423,0.358113,0.357833,0.357685,0.357446,0.357353,0.357192,0.357176,0.357346,0.357559,0.357609,0.357418,0.357786,0.357164,0.35655,0.35668,0.357252,0.357152,0.356461,0.355689,0.355172,0.35503,0.355146,0.355039,0.354686,0.355115,0.35641,0.35769,0.358406,0.358057,0.356689,0.354246,0.351059,0.348097,0.344993,0.341906,0.338854,0.336444,0.334124,0.331551,0.329736,0.329246,0.329379,0.331444,0.332839,0.333569,0.334466,0.334023,0.333909,0.333723,0.334246,0.335027,0.336245,0.337268,0.338525,0.339116,0.338467,0.338372,0.33694,0.336745,0.336803,0.336384,0.336122,0.334923,0.332528,0.331411,0.32907,0.326507,0.323708,0.320999,0.318249,0.315499,0.313189,0.312068,0.312436,0.313858,0.315553,0.317036,0.31803,0.316617,0.311579,0.30943,0.30729,0.306122,0.30592,0.307637,0.308875,0.310349,0.31301,0.314539,0.314522,0.314295,0.313109,0.311587,0.309306,0.310148,0.309006,0.307931,0.307013,0.306457,0.306716,0.307588,0.308264,0.308102,0.306863,0.305061,0.303277,0.301481,0.298899,0.295724,0.291369,0.288974,0.287412,0.288163,0.28812,0.289506,0.294047,0.295919,0.298065,0.298831,0.299819,0.30221,0.3042,0.305055,0.30617,0.308611,0.312365,0.316488,0.319641,0.321086,0.320044,0.316923,0.311844,0.306352,0.299603,0.293879,0.288775,0.283653,0.278094,0.272908,0.268944,0.266015,0.263908,0.262638,0.261471,0.25927,0.255827,0.251402,0.246525,0.241319,0.236598,0.232373,0.229402,0.227611,0.226648,0.226417,0.226331,0.226825,0.227712,0.229002,0.230442,0.231846,0.235785,0.238202,0.239447,0.239381,0.238727,0.237164,0.237469,0.235868,0.233064,0.231769,0.229854,0.228576,0.227749,0.227381,0.227197,0.227321,0.227687,0.228753,0.229435,0.230559,0.232064,0.232515,0.233112,0.233671,0.233637,0.233942,0.233606,0.233244,0.232789,0.23226,0.231887,0.230805,0.230954,0.230577,0.229999,0.229538,0.228811,0.22739,0.226282,0.22499,0.224261,0.22473,0.225412,0.226467,0.226731,0.226514,0.225914,0.225364,0.226134,0.227337,0.228219,0.228426,0.229243,0.229866,0.230366,0.229968,0.229299,0.228608,0.227764,0.227388,0.227685,0.228604,0.22942,0.230047,0.229713,0.229471,0.229024,0.229008,0.228908,0.229019,0.229447,0.229194,0.228901,0.228532,0.22821,0.227819,0.227215,0.22652,0.225629,0.224805,0.22381,0.221749,0.220369,0.220445,0.220946,0.220321,0.219635,0.220242,0.22198,0.223633,0.22546,0.225849,0.226101,0.227064,0.227932,0.228668,0.229103,0.229911,0.23064,0.231144,0.231591,0.232086,0.232443,0.232346,0.232078,0.231833,0.231582,0.231325,0.231076,0.230872,0.230512,0.230354,0.230163,0.2302,0.230074,0.23016,0.229883,0.230113,0.230622,0.230635,0.230319,0.229906,0.229503,0.228785,0.227922,0.227229,0.226609,0.226136,0.225852,0.22558,0.225121,0.224737,0.224403,0.224164,0.224381,0.22479,0.224926,0.224486,0.224076,0.2239,0.223743,0.22346,0.223218,0.222958,0.222832,0.222573,0.221911,0.220798,0.219597,0.219111,0.219304,0.21958]
a = np.array(arr)
avg = np.mean(a)
b = a - avg
c = pd.Series(b)
tol = 0.003
d = c.abs()
#e= d.where(d < tol)
e = d[d<tol]

print(len(e))
#ret = freq_zero_crossing(b, 50)
#ret

7


196    0.001328
197    0.001067
198    0.002629
199    0.001878
200    0.001921
201    0.000535
222    0.001266
dtype: float64

In [0]:
except_arr = ['Unnamed: 0', 'user_id', 'action_id', 'time_50hz', 'user_id_action_id']
window_size = 20

tolerance = 0.003

def generate_mean_zero_count(data):
    stat_list = []  
    statistics_data = {}
    for j in data.columns:
        if j in except_arr:
            continue
        #tdata.iloc[i*window_size:(i+1)*window_size][j]
        #curr_data = data.iloc[i*window_size:(i+1)*window_size][j]
        curr_data = data[j]
        avg = np.average(curr_data)
        shifted_data = curr_data - avg
        pd_shifted_abs_data = pd.Series(shifted_data).abs()
        statistics_data[j+"_zerocount"] = len(pd_shifted_abs_data[pd_shifted_abs_data < tol])
    
    return statistics_data
    

In [0]:
a = pd.DataFrame()
action_id = 'dws_1'
user_id=2
d = data.loc[(data['action_id'].str.match(action_id+"$") ) & (data['user_id'] == user_id)]
ret = generate_mean_zero_count(d)
d.iloc[0]["user_id"]

2

In [0]:
tot = pd.DataFrame()
window_size=20
period_length = 400
label = []
cnt = 1
for action_id in set(data['action_id']):
    print(action_id, "(%d/%d)"%(cnt, len(set(data['action_id']))))
    cnt+=1
    for user_id in set(  data.loc[data['action_id'].str.match(action_id+"$")]['user_id'] ):
        t_data = data.loc[(data['action_id'].str.match(action_id+"$") ) & (data['user_id'] == user_id)]
        ret = generate_mean_zero_count(t_data[:period_length])
        ret["action_id"] = t_data.iloc[0]["action_id"]
        ret["user_id"] = t_data.iloc[0]["user_id"]
        #label += [t_data['action_id']] * period_length
        #print(ret)
        tot = tot.append(ret, ignore_index=True)
tot.to_csv("total_zero_data.csv")
                          

ups_3 (1/15)
dws_1 (2/15)
std_6 (3/15)
jog_16 (4/15)
jog_9 (5/15)
wlk_7 (6/15)
wlk_8 (7/15)
sit_13 (8/15)
std_14 (9/15)
ups_4 (10/15)
dws_2 (11/15)
dws_11 (12/15)
sit_5 (13/15)
wlk_15 (14/15)
ups_12 (15/15)


In [0]:
except_arr = ['Unnamed: 0', 'user_id', 'action_id', 'time_50hz', 'user_id_action_id']
window_size = 100
top_rank_length = 3
stat_list = []

def generate_statistics_data_by_window(tdata, window_size):
    for i in range(int(len(tdata)/window_size)):
        statistics_data = {}
        for j in tdata.columns:
            if j in except_arr:
                continue
            #tdata.iloc[i*window_size:(i+1)*window_size][j]
            curr_data = tdata.iloc[i*window_size:(i+1)*window_size][j]
            statistics_data[j+"_exp"] = np.average(curr_data)
            statistics_data[j+"_median"] = np.median(curr_data)
            statistics_data[j+"_stddev"] = np.std(curr_data)
            statistics_data[j+"_max"] = np.max(curr_data)
            statistics_data[j+"_min"] = np.min(curr_data)
            sort_data = np.sort(curr_data)
            for k in range(1,top_rank_length+1):
                statistics_data[j+"_top_"+(str(k))] = sort_data[-k]
                statistics_data[j+"_bottom_"+(str(k))] = sort_data[k]
        statistics_data["action_id"] = tdata.iloc[0]["action_id"]
        statistics_data["user_id"] = tdata.iloc[0]["user_id"]
        stat_list.append(statistics_data)

    #pd_stat = pd.DataFrame(data=stat_list)
    return stat_list
"""
stat_list = generate_statistics_data_by_window(tdata,window_size)
pd_stat = pd.DataFrame(stat_list)

plt.subplots(figsize=(20, 30))
        
for i in range(1, len(pd_stat.columns)+1):
    plt.subplot(14,5,i)
    plt.title(pd_stat.columns[i-1])
    plt.plot(pd_stat[pd_stat.columns[i-1]])
plt.show()
"""     
        
    
    

'\nstat_list = generate_statistics_data_by_window(tdata,window_size)\npd_stat = pd.DataFrame(stat_list)\n\nplt.subplots(figsize=(20, 30))\n        \nfor i in range(1, len(pd_stat.columns)+1):\n    plt.subplot(14,5,i)\n    plt.title(pd_stat.columns[i-1])\n    plt.plot(pd_stat[pd_stat.columns[i-1]])\nplt.show()\n'

아래와 같이 만들 경우, 데이터가 1Million 건이 넘고, 1GB가 넘기 때문에 reduction을 해야할 것 같다

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import io
import tensorflow as tf

data = pd.read_csv("train_data.csv")
#data = pd.read_csv(io.StringIO(uploaded['train_data.csv'].decode('utf-8')))

tot = pd.DataFrame()
window_size=100
period_length = 400
label = []
cnt=0
for action_id in set(data['action_id']):
    print(action_id, "(%d/%d)"%(cnt, len(set(data['action_id']))))
    cnt+=1
    for user_id in set(  data.loc[data['action_id'].str.match(action_id+"$")]['user_id'] ):
        t_data = data.loc[(data['action_id'].str.match(action_id+"$") ) & (data['user_id'] == user_id)]
        ret = generate_statistics_data_by_window(t_data[:period_length], window_size)
        #label += [t_data['action_id']] * period_length
        #print(ret)
        tot = tot.append(ret)
#tot.to_csv("tot_100_400.csv")

jog_9 (0/15)
std_14 (1/15)
ups_3 (2/15)
jog_16 (3/15)
dws_11 (4/15)
dws_2 (5/15)
sit_13 (6/15)
wlk_8 (7/15)
std_6 (8/15)
ups_12 (9/15)
ups_4 (10/15)
wlk_7 (11/15)
sit_5 (12/15)
dws_1 (13/15)
wlk_15 (14/15)


In [0]:

Y = tot['action_id']          


def classifier(data):
    if data == "wlk":
        return 1
    elif data == "jog":
        return 2
    elif data == "std":
        return 3
    elif data == "sit":
        return 4
    elif data == "dws":
        return 5
    elif data == "ups":
        return 6
    else:
        print("FUCK")
        return -1
Y
Y_translate = list(map(lambda x:x[:3],Y))
Y_translate    
Y_class = list(map(lambda x:classifier(x), Y_translate))
#set(Y_class)
class_count = len(set(Y_class))
Y_one_hot = tf.one_hot(Y_class, class_count)
Y_one_hot.shape

tot2 = tot.drop('user_id', axis=1)
tot2 = tot2.drop('action_id', axis=1)
tot2.sample()

a = np.corrcoef(tot2)
a

NameError: ignored

NameError: ignored

In [0]:
test_data = pd.read_csv("test_data.csv")
#test_data = pd.read_csv(io.StringIO(uploaded['test_data.csv'].decode('utf-8')))

test_pd = pd.DataFrame()
#window_size=100
#period_length = 400
#label = []
cnt=0
for action_id in set(test_data['action_id']):
    print(action_id, "(%d/%d)"%(cnt, len(set(test_data['action_id']))))
    cnt+=1
    for user_id in set(  test_data.loc[data['action_id'].str.match(action_id+"$")]['user_id'] ):
        t_data = test_data.loc[(test_data['action_id'].str.match(action_id+"$") ) & (test_data['user_id'] == user_id)]
        ret = generate_statistics_data_by_window(t_data[:period_length], window_size)
        #label += [t_data['action_id']] * period_length
        #print(ret)
        test_pd = test_pd.append(ret)
#tot.to_csv("tot_100_400.csv")


std_14 (0/15)
dws_1 (1/15)
wlk_7 (2/15)
std_6 (3/15)
wlk_8 (4/15)
dws_11 (5/15)
jog_16 (6/15)
jog_9 (7/15)
sit_13 (8/15)
ups_3 (9/15)
wlk_15 (10/15)
ups_4 (11/15)
ups_12 (12/15)
sit_5 (13/15)
dws_2 (14/15)


In [0]:
test_Y = test_pd['action_id']

test_Y_translate = list(map(lambda x:x[:3],test_Y))
test_Y_class = list(map(lambda x:classifier(x), test_Y_translate))
#set(Y_class)
test_class_count = len(set(test_Y_class))
test_Y_one_hot = tf.one_hot(test_Y_class, test_class_count)
test_Y_one_hot.shape

test_pd2 = test_pd.drop('user_id', axis=1)
test_pd2 = test_pd2.drop('action_id', axis=1)

test_pd2.sample()

,attitude.pitch_bottom_1,attitude.pitch_bottom_2,attitude.pitch_bottom_3,attitude.pitch_exp,attitude.pitch_max,attitude.pitch_median,attitude.pitch_min,attitude.pitch_stddev,attitude.pitch_top_1,attitude.pitch_top_2,...,userAcceleration.z_bottom_2,userAcceleration.z_bottom_3,userAcceleration.z_exp,userAcceleration.z_max,userAcceleration.z_median,userAcceleration.z_min,userAcceleration.z_stddev,userAcceleration.z_top_1,userAcceleration.z_top_2,userAcceleration.z_top_3
283,-1.409588,-1.409577,-1.409504,-1.405218,-1.39935,-1.406009,-1.409649,0.002934,-1.39935,-1.399446,...,-0.014311,-0.013356,0.025537,0.061385,0.028811,-0.024322,0.019346,0.061385,0.055697,0.054669


In [0]:
test_pd2.shape

(20280, 132)

In [0]:
test_Y_one_hot.shape

TensorShape([Dimension(20280), Dimension(6)])

NameError: ignored

In [0]:
learning_rate = 0.01
column_length = len(tot2.columns)
class_count = len(set(Y_class))
x = tf.placeholder(tf.float32, [None, column_length]) #133 column per key (or if you want 20(400/20)*133), set to 133*20)
y = tf.placeholder(tf.float32, [None, class_count], name="Y")
#use tot2

W = tf.Variable(tf.zeros([column_length,class_count], tf.float32))
b = tf.Variable(tf.zeros(class_count))

pred = tf.nn.softmax(tf.matmul(x,W) + b)
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:

    y_one_hot_out = sess.run(Y_one_hot)
    test_y_one_hot_out = sess.run(test_Y_one_hot)

    sess.run(init)

    total_batch = len(tot2)
    batch_size = 400
    epoch_count = 70
    for epoch in range(epoch_count):
        avg_cost=0
        for i in range(int(total_batch / batch_size)):
            batch_xs, batch_ys = tot2.values[i * batch_size : (i+1)*batch_size], y_one_hot_out[i * batch_size : (i+1)*batch_size]
            _,c = sess.run([optimizer, cost], feed_dict={x: batch_xs, y:batch_ys})

            avg_cost += c / total_batch 
        
        
        
        print("-"*50)
        
        print("epoch", "%d / %d"%(epoch, epoch_count), "cost: %.9f"%(avg_cost))
        
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print("test Accuracy:", accuracy.eval({x: test_pd2.values, y:test_y_one_hot_out}))
        
        print("-"*50)

--------------------------------------------------
epoch 0 / 70 cost: 0.001530895
test Accuracy: 0.6449211
--------------------------------------------------
--------------------------------------------------
epoch 1 / 70 cost: 0.001041996
test Accuracy: 0.6650394
--------------------------------------------------
--------------------------------------------------
epoch 2 / 70 cost: 0.000916934
test Accuracy: 0.67332345
--------------------------------------------------
--------------------------------------------------
epoch 3 / 70 cost: 0.000844936
test Accuracy: 0.6851578
--------------------------------------------------
--------------------------------------------------
epoch 4 / 70 cost: 0.000794669
test Accuracy: 0.6940335
--------------------------------------------------
--------------------------------------------------
epoch 5 / 70 cost: 0.000755998
test Accuracy: 0.697288
--------------------------------------------------
--------------------------------------------------
e

있는걸 다 때려박았더니(132개) - accuracy가 0.76정도가 나온다 (epoch 70 정도)

NameError: ignored

132개 다 때려박았더니 train accuracy가 0.64가 나왔다 
왤까?

In [0]:
tot2.values.shape

(995085, 132)

In [0]:
Y_one_hot.shape

TensorShape([Dimension(995400), Dimension(6)])

In [0]:
type(Y_one_hot)

tensorflow.python.framework.ops.Tensor

In [0]:
a = sess.run(Y_one_hot)

In [0]:
a[0:10]

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)

ups_3 (0/15)
dws_1 (1/15)
std_6 (2/15)
jog_16 (3/15)
jog_9 (4/15)
wlk_7 (5/15)
wlk_8 (6/15)
sit_13 (7/15)
std_14 (8/15)
ups_4 (9/15)
dws_2 (10/15)
dws_11 (11/15)
sit_5 (12/15)
wlk_15 (13/15)
ups_12 (14/15)


In [0]:
test_pd

,action_id,attitude.pitch_bottom_1,attitude.pitch_bottom_2,attitude.pitch_bottom_3,attitude.pitch_exp,attitude.pitch_max,attitude.pitch_median,attitude.pitch_min,attitude.pitch_stddev,attitude.pitch_top_1,...,userAcceleration.z_bottom_3,userAcceleration.z_exp,userAcceleration.z_max,userAcceleration.z_median,userAcceleration.z_min,userAcceleration.z_stddev,userAcceleration.z_top_1,userAcceleration.z_top_2,userAcceleration.z_top_3,user_id
0,ups_3,-1.325624,-1.316334,-1.308531,-1.223605,-1.138490,-1.208591,-1.348560,0.068694,-1.138490,...,-0.229259,-0.115666,0.162072,-0.125582,-0.379288,0.140605,0.162072,0.136877,0.068019,1
1,ups_3,-1.393390,-1.381781,-1.358338,-1.258646,-1.136898,-1.258056,-1.407177,0.086575,-1.136898,...,-0.101656,0.033398,0.304778,0.080779,-0.614809,0.257663,0.304778,0.293890,0.287135,1
2,ups_3,-1.313584,-1.312320,-1.303293,-1.220435,-1.159255,-1.190253,-1.324012,0.059597,-1.159255,...,-0.376288,0.002889,0.809307,-0.197411,-0.768190,0.484999,0.809307,0.739095,0.722917,1
3,ups_3,-1.276434,-1.257942,-1.242296,-1.196631,-1.135960,-1.194468,-1.293096,0.046380,-1.135960,...,-0.195390,0.001707,0.252580,0.010576,-0.292159,0.162479,0.252580,0.248019,0.240039,1
4,ups_3,-1.203572,-1.180893,-1.157093,-1.091195,-1.037862,-1.066854,-1.218714,0.056551,-1.037862,...,0.029517,0.215583,0.500378,0.281948,-0.593130,0.280609,0.500378,0.488303,0.480177,1
5,ups_3,-1.181334,-1.177720,-1.173944,-1.126779,-1.071039,-1.125166,-1.183295,0.039935,-1.071039,...,-0.139564,0.083528,0.656738,0.063246,-0.517900,0.287813,0.656738,0.487761,0.404517,1
6,ups_3,-1.162835,-1.139650,-1.130140,-1.076944,-1.010445,-1.073028,-1.194861,0.053908,-1.010445,...,-0.201939,-0.077923,0.346664,-0.091361,-0.493871,0.219201,0.346664,0.305371,0.176016,1
7,ups_3,-1.304746,-1.291599,-1.290984,-1.227132,-1.151130,-1.222613,-1.304825,0.051178,-1.151130,...,-0.075079,0.081667,0.367765,0.139867,-0.606891,0.267349,0.367765,0.338273,0.308756,1
8,ups_3,-1.382685,-1.360877,-1.354641,-1.268509,-1.194337,-1.244727,-1.390301,0.067674,-1.194337,...,-0.374508,0.041225,0.695063,0.057463,-0.640379,0.385650,0.695063,0.681723,0.459457,1
9,ups_3,-1.329548,-1.290551,-1.273338,-1.182007,-1.082810,-1.168753,-1.352929,0.082925,-1.082810,...,-0.276759,-0.126609,0.452750,-0.151486,-0.374106,0.198933,0.452750,0.355071,-0.009225,1
